## Let's Begin

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf

#np.random.seed(19)

In [7]:
train = pd.read_csv('Modified_Training_Data.csv')
test = pd.read_csv('Modified_Testing_Data.csv')
labels = train.pop('Good_Bad')

print('Modified Training data dimensions =', train.shape)
print('Modified Test data dimensions =', test.shape)

Modified Training data dimensions = (2182, 226)
Modified Test data dimensions = (100, 226)


In [8]:
train.drop(['Day_Id'], axis=1, inplace=True)
test_id = test.pop('Day_Id')
train.head()

,Height1 Dew Point,Height1 Pressure,Height1 Temperature,Height1 Wind Direction,Height1 Wind Speed,Height2 Dew Point,Height2 Pressure,Height2 Temperature,Height2 Wind Direction,Height2 Wind Speed,...,Height44 Dew Point,Height44 Pressure,Height44 Temperature,Height44 Wind Direction,Height44 Wind Speed,Height45 Dew Point,Height45 Pressure,Height45 Temperature,Height45 Wind Direction,Height45 Wind Speed
0,-0.119910,0.389446,-0.596675,0.470818,0.198874,-0.028941,0.379991,-0.380792,0.486736,0.338063,...,-0.175333,0.400397,0.096937,0.303945,1.947924,-0.208746,0.404997,0.077952,0.302055,1.916631
1,-0.740140,0.350660,-1.091694,-0.379641,-0.982819,-0.674705,0.329054,-1.087177,-0.372709,-1.129692,...,-1.558844,0.092903,0.003096,-0.119882,-1.442723,-1.412386,0.084670,-0.020079,-0.079968,-1.446260
2,-0.490957,0.662828,-0.958861,0.013198,-0.368329,-0.490240,0.642918,-0.977174,-0.082610,-0.202594,...,0.918063,0.215809,-0.330688,0.030058,-0.897548,0.908338,0.203874,-0.269138,0.211642,-0.924545
3,-0.831845,0.750442,-0.987714,-0.145489,0.746065,-0.813254,0.730137,-1.042170,-0.233787,0.727221,...,-0.566310,0.305596,-0.334611,-1.163486,1.401227,-0.599151,0.294076,-0.336944,-1.183742,1.348804
4,-0.706708,-1.085039,-0.875024,0.001533,0.428631,-0.748520,-1.110612,-0.942177,0.097247,0.470857,...,-0.919724,-1.801882,-1.580280,0.372596,2.245810,-0.940877,-1.821862,-1.595286,0.377003,2.230434


## Building the Neural Network Classifier

In [9]:
# firstly we convert the pandas dataframe to numpy values as tensorflow requires the input to be numpy/tensor array

def data_transform(data, labels):
    data = data.astype(np.float32)
    if labels is not None:
        labels = (np.arange(2) == labels[:,None]).astype(np.float32)
    return data, labels

In [10]:
train, labels = data_transform(train, labels)
test, _ = data_transform(test, None)

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, labels, test_size=0.1, stratify=labels)

## Network Architecture

In [12]:
tf.reset_default_graph()
n0, n1, n2, n3, n4, n5 = 225, 225, 100, 50, 25, 2 

X = tf.placeholder(dtype=tf.float32, shape=[None, n0])

w1 = tf.Variable(tf.truncated_normal(shape=[n0, n1], stddev=0.1))
b1 = tf.Variable(tf.zeros(shape=[n1]))

w2 = tf.Variable(tf.truncated_normal(shape=[n1, n2], stddev=0.1))
b2 = tf.Variable(tf.zeros(shape=[n2]))

w3 = tf.Variable(tf.truncated_normal(shape=[n2, n3], stddev=0.1))
b3 = tf.Variable(tf.zeros(shape=[n3]))

w4 = tf.Variable(tf.truncated_normal(shape=[n3, n4], stddev=0.1))
b4 = tf.Variable(tf.zeros(shape=[n4]))

w5 = tf.Variable(tf.truncated_normal(shape=[n4, n5], stddev=0.1))
b5 = tf.Variable(tf.zeros(shape=[n5]))

In [13]:
y1 = tf.nn.relu(tf.add(tf.matmul(X, w1), b1))
y2 = tf.nn.relu(tf.add(tf.matmul(y1, w2), b2))
y3 = tf.nn.relu(tf.add(tf.matmul(y2, w3), b3))
y4 = tf.nn.relu(tf.add(tf.matmul(y3, w4), b4))

Y = tf.add(tf.matmul(y4, w5), b5)               

In [14]:
Y_true = tf.placeholder(tf.float32, shape=[None, 2])

In [15]:
loss = tf.nn.softmax_cross_entropy_with_logits(logits=Y, labels=Y_true)
mean_loss = tf.reduce_mean(loss)

is_correct = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_true, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

#### Exponentially Decaying Learning Rate

In [16]:
global_step = tf.Variable(0, trainable=False)
start_lr = 0.01
learning_rate = 0.0003
#learning_rate = tf.train.exponential_decay(start_lr, decay_rate=0.9, decay_steps=1000, global_step=global_step, staircase=True)

In [17]:
optimize = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(mean_loss, global_step=global_step)

## Training the NN

In [18]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)

In [19]:
batch_size = 50
num_batches = X_train.shape[0]//batch_size
epoches = 10000
max_acc = -1

for ep in range(epoches):
    start = 0
    end = start + batch_size
    curr_ep_loss = 0
    
    # training the neural network
    for b_no in range(num_batches):
        batch_x, batch_y = X_train[start:end], y_train[start:end]
        start = end

        train_dict = {X: batch_x, Y_true: batch_y}
        _, loss = sess.run([optimize, mean_loss], feed_dict=train_dict)
        curr_ep_loss += loss
        
    curr_ep_loss /= num_batches
    
    # calculating the validation loss per epoch
    # becoz the validation data will not be huge, we could completely feed it without creating batches
    val_dict = {X: X_val, Y_true: y_val}
    val_loss, acc = sess.run([mean_loss, accuracy], feed_dict=val_dict)
    max_acc = max(acc, max_acc)
    '''print("Epoch%3d: Train Loss=%0.4f Val Loss=%0.4f  Val Acc=%0.4f eta=%0.5f global_step=%d"%(
            ep, 
           curr_ep_loss, 
           val_loss,
           acc,
           learning_rate,
           global_step.eval(session=sess)
            ))'''
print('Final Accuracy Achieved =',acc)
print('Maximum Accuracy during training =', max_acc)
#print('Eta =', learning_rate.eval(session=sess))

Final Accuracy Achieved = 0.6621
Maximum Accuracy during training = 0.6621


In [20]:
predict = tf.argmax(tf.nn.softmax(Y),1)
predictions = predict.eval(feed_dict={X: test})

In [21]:
submission = pd.DataFrame({'Day_Id': test_id, 'Label':predictions})
submission.head()

,Day_Id,Label
0,52,0
1,84,1
2,87,0
3,89,0
4,109,1


In [22]:
submission.to_csv('MY_SUBMISSION.csv', index=False)